In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2

In [18]:
from dynamics.model.dms import DMSWrapper
from dynamics.model.pbmp import PBMP
from dynamics.data.datasets.greener.datamodule import GreenerDataModule
from dynamics.utils.loss import rmsd, msd
import torch
import pytorch_lightning as pl

from pytorch_lightning.callbacks import ModelCheckpoint

import hydra
from logging import log
from omegaconf import DictConfig, OmegaConf

In [3]:
hydra.initialize('../config')
hydra.main('../config')

<function hydra.main.main.<locals>.main_decorator(task_function: Callable[[Any], Any]) -> Callable[[], NoneType]>

In [4]:
config = hydra.compose("main")

/home/cch57/.conda/envs/masif/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'main': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [11]:
data_module = GreenerDataModule(config.dataset.dir, config.training.batch_size, config.dataset.fraction)
data = next(iter(data_module.val_dataloader())).to('cuda')

In [9]:
data

DataBatch(x=[112, 24], pos=[112, 3], native_coords=[112, 3], masses=[112], res_numbers=[112, 1], seq=[1], batch=[112], ptr=[2])

In [12]:
model = PBMP(**config.model.params).to('cuda')

In [55]:
checkpoint = '/home/cch57/projects/protein_dynamics/outputs/2022-05-12/16-06-06/dynamics/31otkaea/checkpoints/epoch=0-step=399.ckpt'
checkpoint = torch.load(checkpoint)

checkpoint['state_dict']

state_dict = {}

for key in checkpoint['state_dict']:
    state_dict[key[6:]] = checkpoint['state_dict'][key]


In [53]:
checkpoint = '/home/cch57/projects/protein_dynamics/outputs/2022-05-12/16-06-06/dynamics/31otkaea/checkpoints/epoch=0-step=399.ckpt'
wrapper = DMSWrapper.load_from_checkpoint(checkpoint)


TypeError: __init__() missing 3 required positional arguments: 'input_dim', 'hidden_dim', and 'num_hidden_layers'

In [14]:
with torch.no_grad():
    model.eval()
    out = model(data.pos, data.x, data.res_numbers, data.masses, data.seq)

In [23]:
rmsd(out, data.pos)[0].item(), msd(out, data.pos)[0].item()

(0.4350498914718628, 0.18926841020584106)

In [29]:
with torch.no_grad():
    model2 = model.eval()
    model2.n_steps = 10_000
    out2 = model2(data.pos, data.x, data.res_numbers, data.masses, data.seq, animation=20, animation_steps=10_000)

Saving structure 0 out of 500
Saving structure 1 out of 500
Saving structure 2 out of 500
Saving structure 3 out of 500
Saving structure 4 out of 500
Saving structure 5 out of 500
Saving structure 6 out of 500
Saving structure 7 out of 500
Saving structure 8 out of 500
Saving structure 9 out of 500
Saving structure 10 out of 500
Saving structure 11 out of 500
Saving structure 12 out of 500
Saving structure 13 out of 500
Saving structure 14 out of 500
Saving structure 15 out of 500
Saving structure 16 out of 500
Saving structure 17 out of 500
Saving structure 18 out of 500
Saving structure 19 out of 500
Saving structure 20 out of 500
Saving structure 21 out of 500
Saving structure 22 out of 500
Saving structure 23 out of 500
Saving structure 24 out of 500
Saving structure 25 out of 500
Saving structure 26 out of 500
Saving structure 27 out of 500
Saving structure 28 out of 500
Saving structure 29 out of 500
Saving structure 30 out of 500
Saving structure 31 out of 500
Saving structure 3

In [30]:
from pymol import cmd

ModuleNotFoundError: No module named 'pymol'

In [27]:
rmsd(out2, data.pos)[0].item(), msd(out2, data.pos)[0].item()

(5.848509788513184, 34.2050666809082)